In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from sklearn.preprocessing import StandardScaler
# Load your CSV data using pandas
data = pd.read_csv('classData.csv')

# Combine the values of the first 4 columns into a new column
data['Combination'] = data.apply(lambda row: ''.join(map(str, row[:4])), axis=1)

# Find unique combinations and assign label numbers
unique_combinations = data['Combination'].unique()
label_mapping = {combination: label for label, combination in enumerate(unique_combinations)}

# Add a new column with the label numbers
data['Label'] = data['Combination'].map(label_mapping)

X = data.iloc[:, 4:10].values  # Features
y = data['Label'].values  # Labels

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a function for model training and evaluation
def train_and_evaluate(learning_rate):
    # Build the deep learning model with the given learning rate
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(len(unique_combinations), activation='softmax')
    ])

    custom_optimizer = Adam(learning_rate=learning_rate)

    model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.25, verbose=0)
    
    y_pred = model.predict(X_test)
    y_pred_class = y_pred.argmax(axis=1)
    accuracy = accuracy_score(y_test, y_pred_class)

    return accuracy

# Genetic Algorithm Configuration
population_size = 10
generations = 10
mutation_rate = 0.1
learning_rate_range = (0.0001, 0.1)

# Initialize the population with random learning rates
population = [random.uniform(*learning_rate_range) for _ in range(population_size)]
fitness_scores = [0] * population_size  # Initialize with zeros

for generation in range(generations):
    # Evaluate the fitness of each individual in the population
    for i, learning_rate in enumerate(population):
        fitness_scores[i] = train_and_evaluate(learning_rate)

    # Select the top performers (elite)
    elite_count = int(population_size * 0.2)
    elite_indices = np.argsort(fitness_scores)[-elite_count:]

    # Create a new population with elite individuals and mutated individuals
    new_population = [population[i] for i in elite_indices]

    # Perform mutation
    for i in range(population_size - elite_count):
        if random.random() < mutation_rate:
            # Mutate by generating a new random learning rate
            new_population.append(random.uniform(*learning_rate_range))
        else:
            # Select a parent from the elite individuals
            parent_index = random.choice(elite_indices)
            new_population.append(population[parent_index])

    population = new_population

# Find the best learning rate in the final population
best_learning_rate = population[np.argmax(fitness_scores)]

print(f"Best Learning Rate: {best_learning_rate:.5f}")

#Corresponding accuracy of best learning rate model is
best_accuracy = max(fitness_scores)



50/50 [==============================] - 0s 946us/step
Best Learning Rate: 0.06394


In [4]:
print(f"Accuracy: {best_accuracy:.2%}")

Accuracy: 85.76%
